In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import KFold

In [2]:
df = pd.read_pickle('Complete_NST_Text.pkl')

#cleans \n slashes from text
df['text'] = df['text'].replace(r'\n', ' ', regex=True)

In [13]:
df.head(50)

,Title,Date,"Community (city,town, ward)",LGA,State,Total Deaths,Boko Haram (P),State Actor (P),Sectarian Actor (excluding BH) (P),Other Armed Actor (P),...,Notes,Sources 1,Sources 2,Sources 3,Latitude,Longitude,full place name,country,text,Boko_Haram_code
0,Army Barracks Attack Ahead of Goodluck Jonatha...,5/29/2011,Bauchi,Bauchi,Bauchi,20,Boko Haram,NaN,NaN,NaN,...,NaN,http://allafrica.com/stories/201105310776.html,http://www.reuters.com/article/2011/05/30/us-n...,http://www.vanguardngr.com/2011/06/boko-haram-...,#ERROR!,#ERROR!,"Bauchi, Bauchi, Nigeria",Nigeria,"BAUCHI, Nigeria (Reuters) - A bomb blast rocke...",1
1,Kaduna,5/29/2011,Federal Capital Territory,Zaria,Kaduna,6,Boko Haram,NaN,NaN,NaN,...,NaN,http://weeklytrust.com.ng/?option=com_content&...,http://allafrica.com/stories/201105311096.html,http://allafrica.com/stories/201105310901.html,#ERROR!,#ERROR!,"Zaria, Kaduna, Nigeria",Nigeria,Articles Top 10 Hottest Nigerian Music Artist...,1
2,Morning Attack in Zaria,5/30/2011,"Iyan Juma, Zaria City",Zaria,Kaduna,2,Boko Haram,NaN,NaN,NaN,...,NaN,http://weeklytrust.com.ng/?option=com_content&...,http://weeklytrust.com.ng/?option=com_content&...,http://weeklytrust.com.ng/?option=com_content&...,#ERROR!,#ERROR!,"Zaria, Kaduna, Nigeria",Nigeria,Articles Top 10 Hottest Nigerian Music Artist...,1
5,Militants Kill Muslim Cleric Ibrahim Burkuti,6/6/2011,Biu,Biu,Borno,1,Boko Haram,NaN,NaN,NaN,...,NaN,http://www.bbc.co.uk/news/world-africa-13724349,http://www.iol.co.za/news/africa/radical-cleri...,"BBC, AFP",NaN,NaN,"Biu, Borno, Nigeria",Nigeria,Image caption Three people were killed when an...,1
6,Bomb Blasts Outside St. Patrick's Catholic Church,6/7/2011,Maiduguri,Maiduguri,Borno,5,Boko Haram,NaN,NaN,NaN,...,NaN,http://allafrica.com/stories/201106080424.html,http://www.vanguardngr.com/2011/06/carnage-the...,http://news.xinhuanet.com/english2010/world/20...,NaN,NaN,"Maiduguri, Borno, Nigeria",Nigeria,"In January 2010, the group struck again in Bor...",1
7,Bombing of Kano Motor Park,6/7/2011,Kano Motor Park,Maiduguri,Borno,0,Boko Haram,NaN,NaN,NaN,...,NaN,http://allafrica.com/stories/201106080755.html,http://www.panapress.com/11-killed-in-multiple...,NaN,NaN,NaN,"Maiduguri, Borno, Nigeria",Nigeria,11 killed in multiple bombings in Nigerian cit...,1
8,David Usman and Shot Dead,6/7/2011,Maiduguri,Maiduguri,Borno,2,Boko Haram,NaN,NaN,NaN,...,NaN,http://allafrica.com/stories/201106100373.html,http://www.bbc.co.uk/news/world-africa-13724349,NaN,NaN,NaN,"Maiduguri, Borno, Nigeria",Nigeria,Image caption Three people were killed when an...,1
12,Attack on Nigeria's Police HQ,6/16/2011,"Police HQ, Louis Edet House",Abuja Municipal Area Council (AMAC),Federal Capital Territory,2,Boko Haram,NaN,NaN,NaN,...,NaN,http://dailytrust.com.ng/index.php?option=com_...,http://www.telegraph.co.uk/news/worldnews/afri...,NaN,#ERROR!,#ERROR!,"Abuja Municipal Area Council (AMAC), Federal C...",Nigeria,Thick black smoke rose into the sky above the ...,1
13,Bank Assault in Katsina,6/20/2011,Katsina,Kankara,Katsina,already counted,NaN,NaN,NaN,NaN,...,NaN,http://allafrica.com/stories/201106210508.html,http://www.bbc.co.uk/news/world-africa-13861739,NaN,NaN,NaN,"Kankara, Katsina, Nigeria",Nigeria,Image copyright AFP Image caption Boko Haram t...,0
14,Gunmen Kill Police Officer in Maiduguri,6/20/2011,"Lawan Bukar Ward, Maiduguri",Maiduguri,Borno,1,Boko Haram,NaN,NaN,NaN,...,NaN,http://abcnews.go.com/International/wireStory?...,http://allafrica.com/stories/201106240890.html,NaN,NaN,NaN,"Maiduguri, Borno, Nigeria",Nigeria,Dozens of Syrian militants and their families ...,1


In [ ]:
#check Boko Haram

In [4]:
def set_cat(x, word):
    if x == word:
        return 1
    else:
        return 0

In [5]:
df['Boko_Haram_code'] =df['Boko Haram (P)'].apply(lambda x: set_cat(x, 'Boko Haram'))

In [10]:
def log_reg(x,y):
    X_train, X_test, y_train, y_test = train_test_split(x, y)
    #creates a count vectorizer object
    vect = CountVectorizer(stop_words='english').fit(X_train)

    #uses vect object to create a sparse matrix
    X_train_vectorized = vect.transform(X_train)

    #creates and fits model
    model = LogisticRegression(solver='newton-cg', multi_class='multinomial')
    model.fit(X_train_vectorized, y_train)



    predictions = model.predict(vect.transform(X_test))
    return accuracy_score(predictions, y_test)
    

In [11]:
log_reg(df['text'], df['Boko_Haram_code'])

0.90026954177897578

In [12]:
#check state actor

In [14]:
df['state_actor_code'] = df['State Actor (P)'].apply(lambda x: set_cat(x, 'State Actor'))

In [18]:
log_reg(df['text'], df['state_actor_code'])

0.85624438454627139

In [19]:
#check sectarian actor

In [20]:
df['sect_code'] =df['Sectarian Actor (excluding BH) (P)'].apply(lambda x: set_cat(x, 'Sectarian Actor'))

In [21]:
log_reg(df['text'], df['sect_code'])

0.92902066486972146

In [22]:
#other armed actor

In [24]:
df['other_armed_actor_code'] = df['Other Armed Actor (P)'].apply(lambda x: set_cat(x, 'Other Armed Actor'))

In [26]:
log_reg(df['text'], df['other_armed_actor_code'])

0.85175202156334229

In [27]:
#deaths

In [33]:
df['total_deaths_code'] = df['Total Deaths'].apply(lambda x: convert_num(x))

In [31]:
def convert_num(x):
    try:
        return int(x)
    except:
        return 0

In [35]:
from sklearn.linear_model import LinearRegression

In [42]:
def lin_reg(x,y):
    X_train, X_test, y_train, y_test = train_test_split(x, y)
    #creates a count vectorizer object
    vect = CountVectorizer(stop_words='english').fit(X_train)

    #uses vect object to create a sparse matrix
    X_train_vectorized = vect.transform(X_train)

    #creates and fits model
    model = LinearRegression()
    model.fit(X_train_vectorized, y_train)
    return model.score(vect.transform(X_test), y_test)



#     predictions = model.predict(vect.transform(X_test))
#     return accuracy_score(predictions, y_test)

In [43]:
lin_reg(df.text, df.total_deaths_code)

-0.0022846902139925795

0       BAUCHI, Nigeria (Reuters) - A bomb blast rocke...
1       Articles  Top 10 Hottest Nigerian Music Artist...
2       Articles  Top 10 Hottest Nigerian Music Artist...
5       Image caption Three people were killed when an...
6       In January 2010, the group struck again in Bor...
7       11 killed in multiple bombings in Nigerian cit...
8       Image caption Three people were killed when an...
12      Thick black smoke rose into the sky above the ...
13      Image copyright AFP Image caption Boko Haram t...
14      Dozens of Syrian militants and their families ...
15      Image copyright AFP Image caption Boko Haram t...
18      By Samuel Oyadongha  Yenagoa—Tragedy struck, y...
19      AP, MAIDUGURI, NIGERIA  Men riding motorcycles...
20      MAIDUGURI, Nigeria — Suspected members of a ra...
22      *Al-Qaeda plans base in Nigeria, British secur...
25      An explosion and heavy gunfire has rocked the ...
29      MAIDUGURI, Nigeria, July 10 (UPI) -- More than...
30      KADUNA